# 函数准备

In [2]:
import os
import numpy as np
import pandas as pd
import json
from copy import deepcopy
import re
import random
from random import choice
random.seed(10)
# -*- coding:utf-8 -*
import eval
from mkdir import mk_dir
from bert_format import bset,test_qa

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 10)
# for col_ab in ["age", "gender","num","evi","immu"]:
for col_ab in ['pat']:

    mk_dir("eval/evl_%s" % (col_ab))

    train_set = json.load(open("input/input_%s/train_set.json" \
        %(col_ab), "r"))
    train_qa = train_set['data'][0]["paragraphs"]

    re = 0
    for ttrain_qa,eval_qa in kf.split(train_qa):

        re = re + 1
        bset(
            [train_qa[i] for i in ttrain_qa],
            "eval/evl_%s/input_%s/" % (col_ab, re),
            "train_set.json")
        
        eval_q=[]
        eval_a={}
        for i in eval_qa:
            
            eval_q.append(test_qa(
                train_qa[i]['qas'][0]['id'],
                train_qa[i]['qas'][0]['question'],
                train_qa[i]['context']))
            eval_a[train_qa[i]['qas'][0]['id']] = train_qa[i]['qas'][0]['answers']
            
        bset(
            eval_q, 
            "eval/evl_%s/output_%s/" % (col_ab, re), 
            "test_set.json")
        json.dump(eval_a, open("eval/evl_%s/input_%s/eval_set.json" 
        % (col_ab, re), "w"))

In [ ]:
col_ab =  "pat"
standard_0 = pd.DataFrame(np.zeros((0,1)), columns = ["golden"])
prediction_0 = pd.DataFrame(np.zeros((0,3)), columns = ["prediction","rate","cut"])
for re in range(1,11):
    eval_a = json.load(open("eval/evl_%s/input_%s/eval_set.json" % (col_ab, re), "r"))
    standard = pd.DataFrame(eval_a, index=["golden"]).T
    standard_0 = pd.concat((standard_0, standard), 0)
    prediction = json.load(open("eval/evl_%s/output_%s/predictions_.json" % (col_ab, re),'r'))
    prediction = pd.DataFrame(prediction, index=["prediction", "rate"]).T
    prediction["cut"] = 1
    for idx in prediction.index:
        prediction.loc[idx, "rate"] = prediction.loc[idx, "rate"][0]
        prediction.loc[idx, "cut"] = 1 - prediction.loc[idx, "rate"]
        if prediction.loc[idx, "rate"] < 0.85:
            prediction.loc[idx, "prediction"] = "no"
    prediction_0 = pd.concat((prediction_0, prediction),0)

fig_path = ("eval/evl_%s/output_%s" % (col_ab, re))
eval.evaluate(standard_0, prediction_0, fig_path)

# 关键词

In [40]:
pneumonia = {"bronchitis","bronchopneumonia","pneumonia","pneumonitis","pneumonic","pleuropneumonia","pleural"}
LungRelated = {"bronchus","bronchial","tracheobronchial","bronchiolar",      "bronchopulmonary",
               "bronchoalveolar","bronchoscopy",
               "lung","pulmonary","pneumonic","bronchopulmonary","respiratory","respiration"}
infection = {"infection","infections","abscess","pathology","etiology",  "diseases",
             "bacteremia","sepsis","septic","pathogenicity","legionellosis","empyema","necrosis"}

In [ ]:
df = pd.read_excel("/home/data/t070224/Pubmed/Pubmed_Label/3.Orgnize/output/文献合并/human_label_pat.xlsx")
df['golden'] = 'no'
df.loc[df["是否是致病菌"]=='yes', "golden"] = 'yes'
df["full"] = df["Full"].str.lower()
df["prediction"] = 'no'
for aword in pneumonia:
    df.loc[df["full"].str.contains(aword), "prediction"] = 'yes'
    print(aword, df.loc[df["full"].str.contains(aword), ['golden','prediction', 'Full','原文链接']])
for aword in LungRelated:
    for bword in infection:
        df.loc[(df["full"].str.contains(aword)) & (df["full"].str.contains(bword)) , "prediction"] = 'yes'
        print(aword, df.loc[df["full"].str.contains(aword), ['golden','prediction', 'Full','原文链接']])
fig_path = ''
eval.evaluate(df[['golden']], df[['prediction']], fig_path)

# 免疫状态

In [4]:
df = pd.read_excel("/home/data/t070224/Pubmed/Pubmed_Label/3.Orgnize/output/文献合并/human_label_pat.xlsx")
df = df.loc[df["是否是致病菌"]=='yes', :]
df['golden'] = 'no'
df.loc[df["免疫状态"]=='抑制', "golden"] = 'yes'
df["full"] = df["Full"].str.lower()
df["prediction"] = 'no'
for aword in [
    # "immunodeficiency", 
    "immunocompromised", "immunosuppressive","immunosuppression", 
    # "compromised", "suppressed", "supperssive", "suppression",
    "hiv","hiv-positive","aids",
    "transplantation", "transplanted",
    "malignancy", "cancer", "carcinoma", 
    "leukemia","leukaemia", "lymphoma", "melanoma", "myeloma",
    "impair", "impaired immune", "fibrosis"]:
    df.loc[df["full"].str.contains(aword), "prediction"] = 'yes'
    # print(aword, df.loc[df["full"].str.contains(aword), ['golden','prediction', 'Full','原文链接']])
for aword in ["immunocompetent"]:
    df.loc[df["full"].str.contains(aword), "prediction"] = 'no'


In [6]:
eval.evaluate(df[['golden']], df[['prediction']])

结果: 0.7985347985347986 0.651685393258427 0.7073170731707317 0.6783625730994152


(0.7985347985347986, 0.651685393258427, 0.7073170731707317, 0.6783625730994152)

# 预后

In [ ]:
df = pd.read_excel("/home/data/t070224/Pubmed/Pubmed_Label/3.Orgnize/output/文献合并/human_label_pat.xlsx")
df = df.loc[df["是否是致病菌"]=='yes', :]
df['golden'] = 'no'
df.loc[df["预后"]=='死亡', "golden"] = 'yes'
df["full"] = df["Full"].str.lower()
df["prediction"] = 'no'
for aword in [
    'death', 'dead']:
    df.loc[df["full"].str.contains(aword), "prediction"] = 'yes'
    print(aword, df.loc[df["full"].str.contains(aword), ['golden','prediction', 'Full','原文链接']])

In [39]:
fig_path = ''
eval.evaluate(df[['golden']], df[['prediction']], fig_path)

结果: 0.8992673992673993 0.5161290322580645 0.2857142857142857 0.367816091954023


/home/data/t070224/Pubmed/Pubmed_Bert/eval.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  standard.loc[standard["golden"] == "yes", "golden"] = 1
/home/data/t070224/Pubmed/Pubmed_Bert/eval.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  standard.loc[standard["golden"] == "no", "golden"] = 0
/home/data/t070224/Pubmed/Pubmed_Bert/eval.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

KeyError: "None of [Index(['rate', 'cut'], dtype='object')] are in the [columns]"